In [60]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [61]:
sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=10)).strftime('%Y-%m-%d') #1 months
sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime('%Y-%m-%d') #1 months

In [63]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165821,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [64]:
query = """
        SELECT DISTINCT
            bs.site_identifier_value AS city,
            bo.offer_id,
            bo.offer_name as discount,
            DATE(bo.start_date) AS start_date

        FROM postgres_broadleaf_federate."broadleaf.blc_offer" bo
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref" bocsx on (bo.offer_id = bocsx.offer_id and ((bocsx.archived is null or bocsx.archived='N')))
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_customer_offer_xref" bcox on (bo.offer_id = bcox.offer_id  )
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref" btcofx on (btcofx.offer_id = bo.offer_id)
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria" boic on (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" bs on bs.site_id = bo.catalog_disc
        
        WHERE (end_date > current_date - 15 or end_date IS NULL)
            and start_date <= current_date + 2
            and bo.offer_discount_type = 'PERCENT_OFF'
            and bo.offer_type = 'ORDER_ITEM'
            and bo.sndbx_id is null
            and (bo.archived is null or bo.archived='N')
            and (bo.sndbx_tier is null or bo.sndbx_tier = 999999)
            and bo.automatically_added = 'true'
            and (boic.sndbx_tier is null or bo.sndbx_tier = 999999)
            and (bocsx.sndbx_tier is null or bocsx.sndbx_tier = 999999)
            and bo.offer_name NOT ILIKE '%merma%'
            AND offer_name NOT ILIKE 'ACMKTPLC%'
            AND offer_name NOT ILIKE '%\_NB\_%'
            AND offer_name NOT ILIKE '%BIG LIQUIDATION%'
            AND (offer_name ILIKE 'SH\_GRADS%' OR offer_name ILIKE 'SH\_FARM%')
"""
df_offers = read_connection_data_warehouse.runQuery(query)

In [65]:
discount_counts = df_offers.groupby('start_date').size()
discount_counts

start_date
2023-07-10      546
2023-07-13     5391
2023-07-14    12577
dtype: int64

In [67]:
query = """
SELECT 
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    --fs.order_close_date AS close_date,
    --fs.order_id,
    dc.source_id AS customer_id,
    --cat.parent_description as category,
    --cat.description AS subcategory,
    dp.card_id,
    --dp.source_id AS sku_id,
    dp.card_description AS product_name,
    --fs.product_quantity_x_step_unit AS cant,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local END AS gmv_usd,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
        WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount END AS discount_applied,
    fsd.adjustment_reason--,
    --bo.source_id AS offer_id
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    AND (fsd.adjustment_reason ILIKE 'SH\_GRAD%' OR fsd.adjustment_reason ILIKE 'SH\_FARM%')
""".format(Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin)

df = run_read_dwd_query(query)

In [68]:
grouped_data = df.groupby('submit_date')['customer_id'].agg([('customer_tuple', lambda x: tuple(set(x))), ('customer_count', 'count')])

In [69]:
grouped_data

,customer_tuple,customer_count
submit_date,,
2023-07-10,"(23099392, 174299911, 180357260, 104318866, -2...",46
2023-07-11,"(46655104, 59200640, 216760333, 61400717, 1676...",109
2023-07-12,"(-235551, 171816163, 49854820, 221141477, 1834...",16
2023-07-13,"(-266236, 30347285, 62750742, 44339223, 168878...",998
2023-07-14,"(44220418, 42629133, 172265487, 200026135, 146...",1743
2023-07-15,"(105850882, 67995660, 209844239, 16308244, 216...",679
2023-07-16,"(18184195, 264728324, 101054341, 15238534, 157...",101
2023-07-17,"(103072769, 139238410, 138018833, 100337682, -...",304


In [73]:
num = 4
query = """
SELECT 
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    --fs.order_close_date AS close_date,
    --fs.order_id,
    dc.source_id AS customer_id,
    --cat.parent_description as category,
    --cat.description AS subcategory,
    dp.card_id,
    --dp.source_id AS sku_id,
    dp.card_description AS product_name,
    --fs.product_quantity_x_step_unit AS cant,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local END AS gmv_usd,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
        WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount END AS discount_applied,
    CASE
        WHEN fsd.adjustment_reason ILIKE 'SH\_GRAD%' THEN 'EXP'
        WHEN fsd.adjustment_reason IS NULL THEN 'NO DISCOUNT'
    ELSE 'OTHERS DCT' END AS responsable,
    fsd.adjustment_reason--,
    --bo.source_id AS offer_id
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND DATE(fs.order_submitted_date) = '{Fecha_start}'
    AND dc.source_id IN {customers}
""".format(Fecha_start = grouped_data.index[num].strftime('%Y-%m-%d'), customers= grouped_data.values[num][0])

df = run_read_dwd_query(query)
df[["gmv_usd", "discount_applied"]] = df[["gmv_usd", "discount_applied"]].astype('float32')

df = pd.merge(df, df_margin[['margin','card_id']], left_on='card_id', right_on='card_id', how='inner')

df["cash_margin"] = df.gmv_usd * df.margin
df["net_cash_margin"] = df.cash_margin - df.discount_applied


In [74]:
# Pivot table for GMV
pivot_gmv = pd.pivot_table(df, values='gmv_usd', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for discount
pivot_discount = pd.pivot_table(df, values='discount_applied', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for cash_margin
pivot_cash = pd.pivot_table(df, values='cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for net_cash_margin
pivot_net_cash = pd.pivot_table(df, values='net_cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

In [75]:
# Concatenate the pivot tables side by side
comparison_table = pd.concat([pivot_cash, pivot_net_cash], axis=1)

# Display the comparison table
comparison_table

responsable,EXP,NO DISCOUNT,OTHERS DCT,Total,EXP,NO DISCOUNT,OTHERS DCT,Total
customer_id,,,,,,,,
-254753,8.325748,0.286338,7.906129,16.518214,1.375287,0.0,5.987871,7.363158
-254194,2.697189,-0.182353,5.432731,7.947567,-1.379373,0.0,2.958049,1.578676
-252514,7.903025,2.471068,11.045317,21.419409,4.880474,0.0,6.209689,11.090163
-252117,2.906695,13.333335,12.565021,28.805050,2.115302,0.0,-12.174061,-10.058759
-251520,0.758513,0.581991,1.435110,2.775614,0.549654,0.0,0.233346,0.783001
...,...,...,...,...,...,...,...,...
348434107,0.462961,0.335048,0.536147,1.334157,0.100975,0.0,0.212270,0.313245
348521353,0.925923,8.609545,11.777263,21.312730,0.201950,0.0,7.482614,7.684564
348534116,0.169161,2.549490,1.760116,4.478766,-0.124987,0.0,1.291923,1.166936


In [49]:
pivot_cash

responsable,EXP,NO DISCOUNT,OTHERS DCT,Total
customer_id,,,,
-251867,5.856304,31.338049,40.238531,77.432884
-243694,1.230816,1.451861,7.805527,10.488204
-240977,1.105857,6.993956,19.696843,27.796656
-238932,0.967224,6.781460,5.764651,13.513335
15795931,5.454193,2.804446,3.055698,11.314337
23099392,5.038322,0.661649,NaN,5.699971
32393784,0.977389,58.271202,12.492029,71.740620
40183186,2.318320,1.128248,2.068483,5.515051
42141048,2.017297,2.486782,2.522070,7.026149


In [24]:
pivot_discount

responsable,EXP,NO DISCOUNT,OTHERS DCT,Total
customer_id,,,,
-251867,1.253707624,0,7.570905711,8.824613335
-243694,0.818802966,0,14.307822032,15.126624998
-240977,1.268538135,0,4.828866523,6.097404658
-238932,0.551800845,0,0.681181138,1.232981983
15795931,3.238781778,0,2.946085804,6.184867582
23099392,5.129136652,0,NaN,5.129136652
32393784,0.289194915,0,1.006488345,1.295683260
40183186,1.789904660,0,0.802844278,2.592748938
42141048,1.408527540,0,2.090090041,3.498617581
